# 1D implantation simulations

You can call install [PySRIM](https://pypi.org/project/pysrim/) to perform implant simulations with `pip install pysrim`

Thanks to this plugin you can directly interface with the implant layers of your Components for drift-diffusion simulations and use the results of the ion implantation simulations

[SRIM is a Visual Basic GUI program](http://www.srim.org/) which has been thoroughly benchmarked.
The website contains lots of documentation on the physics and software.
It is closed source, but is free to use, copy, modify and distributed for any non-commercial purpose.
To install it, you can follow the instructions on the [PySRIM repository](https://gitlab.com/costrouc/pysrim/).
You can install the Windows executable yourself (using Wine on MacOS/Linux), or use a Docker image.
[The issues contain good information if you run into problems.](https://gitlab.com/costrouc/pysrim/-/issues/7)

In [ ]:
%matplotlib inline

import os
from pprint import pprint

# Only needed to embed the images in notebook
from IPython.display import Image

import numpy as np
import matplotlib.pyplot as plt

from srim import TRIM, SR, Ion, Layer, Target
from srim.output import Results

[The following example follows the tutorial from PySRIM](https://gitlab.com/costrouc/pysrim/-/blob/master/examples/notebooks/Analysis.ipynb), adapted for silicon photonic applications.

## Simulating n-doping of silicon

### Setup

Let's compute the implant depth for 100 keV Phosphorus (a typical N-implant) into 220-nm thick SOI:

In [ ]:
# Define implant
energy = 1.0e5
implant = Ion("P", energy=1.0e5)

# Define layers of target
nm = 10  # units of SRIM are Angstroms
um = 1e4
soi_thickness = 220 * nm
BOX_thickness = 100 * nm  # instead of 3 * um, ions barely make it to BOX

# 220nm pure silicon
soi = Layer(
    {
        "Si": {
            "stoich": 1.0,  # (float, int, required): Stoichiometry of element (fraction)
            "E_d": 35.0,  # (float, int, optional): Displacement energy [eV]
            "lattice": 0.0,  # (float, int, optional): Lattice binding energies [eV]. Used for sputtering calculations.
            "surface": 3.0,  # (float, int, optional): Surface binding energies [eV]. Used for sputtering calculations.
        },
    },
    density=2.3290,  # density [g/cm^3] of material
    width=soi_thickness,  # width [Angstroms] of layer
)

# 3um SiO2
box = Layer(
    {
        "Si": {
            "stoich": 0.33,
        },
        "O": {
            "stoich": 0.67,
        },
    },
    density=2.65,
    width=BOX_thickness,
)

# Define multilayer target
target = Target([soi, box])

### Executing simulation

PySRIM recommends fragmenting the simulation to avoid SRIM crashing:

In [ ]:
from pathlib import Path
from shutil import rmtree
from itertools import count

# Helper functions to run calculations
def fragment(step, total):
    remaining = total
    while remaining > 0:
        if step > remaining:
            return remaining
        else:
            remaining -= step
            yield step


def find_folder(directory):
    for i in count():
        path = Path(directory) / str(i)
        if not path.is_dir():
            return str(path.absolute())


def run_fragmented_calculation(ion, target, number_ions, path, trim_settings, step):
    for i, num_ions in enumerate(fragment(step, number_ions)):
        print(
            "total ions completed: {:06d}\tion: {}\tions in step: {:06d}".format(
                i * step, ion.symbol, num_ions
            )
        )
        trim_settings = trim_settings or {"calculation": 2}
        trim = TRIM(target, ion, number_ions=num_ions, **trim_settings)
        results = trim.run(srim_executable_directory)
        save_directory = find_folder(path)
        print("calculation saved to:", save_directory)
        os.makedirs(save_directory, exist_ok=True)
        TRIM.copy_output_files(srim_executable_directory, save_directory)

In [ ]:
overwrite = True

srim_executable_directory = Path("/tmp/srim")
srim_data_directory = Path("./tmp/")

srim_data_directory.mkdir(exist_ok=True, parents=True)
srim_executable_directory.mkdir(exist_ok=True, parents=True)

trim_settings = {
    "calculation": 1,
    "angle_ions": 7,
    "ranges": True,
    "plot_mode": 5,
}

if overwrite:
    rmtree(srim_data_directory)
    trim = run_fragmented_calculation(
        ion=implant,
        target=target,
        number_ions=5000,
        path=srim_data_directory,
        trim_settings=trim_settings,
        step=1000,
    )

If you are using your own TRIM installation, you should see a window popup and run the calculations. If using Docker, the process will hang until it is done (there is no progress monitor).

### Analyzing vertical distribution

Now we can analyze the results. We are interested in the implant distribution, and so we plot the range of the ions in the target. First, we collect the different results and normalize the results:

In [ ]:
results = [
    Results(child)
    for child in srim_data_directory.glob("**/")
    if child != srim_data_directory
]

In [ ]:
depths = np.mean(np.vstack([result.range.depth for result in results]), axis=0)
ranges = np.mean(
    np.vstack([result.range.ions / result.range.num_ions for result in results]),
    axis=0,
)
ranges = ranges / np.sqrt(np.trapz(ranges, depths))  # normalize distribution

In [ ]:
plt.title(f"Phosphorus {energy} eV in SOI")
plt.scatter(depths, ranges, marker=".", color="k")

plt.xlabel("Depth (A)")
plt.ylabel("Relative number of ions")

plt.vlines(x=soi_thickness, ymin=0, ymax=1000)
plt.xlim([0, 4000])
plt.ylim([0, 0.35])

We can do statistical analysis:

In [ ]:
from statistics import mean, stdev
from scipy.stats import kurtosis, skew

implant_range = mean(ranges * depths)
implant_straggle = stdev(ranges * depths)
implant_skew = skew(ranges * depths)
implant_kurtosis = kurtosis(ranges * depths)

import pprint

pprint.pprint(f"Implant range: {implant_range}")
pprint.pprint(f"Implant straggle: {implant_straggle}")
pprint.pprint(f"Implant skew: {implant_skew}")
pprint.pprint(f"Implant kurtosis: {implant_kurtosis}")